## OpenFace

In [1]:
import numpy as np
import imutils
import pickle
import time
import cv2
import os

# Paths to model files and recognizer
embeddingModel = r"D:\PythonProject\Face_Recognition_DL\openface.nn4.small2.v1.t7"
recognizerFile = r"D:\PythonProject\Face_Recognition_DL\output\recognizer_updated_Openface01.pickle"
labelEncFile = r"D:\PythonProject\Face_Recognition_DL\output\le_updated_Openface01.pickle"
conf_threshold = 0.8  # Confidence threshold

# Paths to the face detection model (SSD)
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"
detection_threshold = 0.5

# Load models
print("[INFO] Loading face detection and recognition models...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
embedder = cv2.dnn.readNetFromTorch(embeddingModel)
recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

# Initialize camera stream
print("[INFO] Starting video stream...")
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
time.sleep(2.0)

# Create directory for recognized faces
if not os.path.exists("recognized_faces"):
    os.makedirs("recognized_faces")

while True:
    ret, frame = cam.read()
    if not ret:
        continue
    
    frame = imutils.resize(frame, width=640)  # Resize for performance
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    faceBlobs, boxes = [], []
    
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < detection_threshold:
            continue
        
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        if startX < 0 or startY < 0 or endX > w or endY > h:
            continue
        
        face = frame[startY:endY, startX:endX]
        if face.shape[0] == 0 or face.shape[1] == 0:
            continue
        
        faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        faceBlobs.append(faceBlob)
        boxes.append((startX, startY, endX, endY))
    
    if len(faceBlobs) > 0:
        faceBlobs = np.vstack(faceBlobs)
        embedder.setInput(faceBlobs)
        vecs = embedder.forward()
        
        for i, vec in enumerate(vecs):
            preds = recognizer.predict_proba([vec.flatten()])[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j] if proba * 100 >= 65 else "Unknown"
            
            (startX, startY, endX, endY) = boxes[i]
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 1)
            cv2.putText(frame, f"{name}: {proba*100:.2f}%", (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)
            
            if name != "Unknown":
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                cv2.imwrite(f"recognized_faces/{name}_{timestamp}.jpg", frame[startY:endY, startX:endX])
    
    cv2.imshow("Frame", frame)
    
    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) < 1:  # Close button clicked
        break
    
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC key to quit
        break

# Release resources
cam.release()
cv2.destroyAllWindows()


[INFO] Loading face detection and recognition models...


d:\PythonProject\Face_Recognition_DL\HogEnv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\PythonProject\Face_Recognition_DL\HogEnv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[INFO] Starting video stream...


## Arc Face

In [5]:
#ArcFace
import numpy as np
import pickle
import time
import cv2
import torch
from ultralytics import YOLO
import imutils

# Paths to model files and recognizer
recognizerFile = r"D:\PythonProject\Face_Recognition_DL\output\recognizer_updated_arc.pickle"
labelEncFile = r"D:\PythonProject\Face_Recognition_DL\output\le_updated_arc.pickle"
embeddingFile = r"D:\PythonProject\Face_Recognition_DL\output\embeddings_arcface.pickle"
yolo_model_path = r"D:\PythonProject\Face_Recognition_DL\model\yolov8n-face.pt"
conf = 0.8  # Confidence threshold

# Load YOLO model for face detection
print("[INFO] Loading YOLO model...")
yolo = YOLO(yolo_model_path)

# Load the face recognizer and label encoder
print("[INFO] Loading face recognizer...")
recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

# Load saved embeddings
print("[INFO] Loading stored face embeddings...")
data = pickle.loads(open(embeddingFile, "rb").read())
known_embeddings = np.array(data["embeddings"])
known_names = np.array(data["names"])

# Initialize camera stream
print("[INFO] Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

def get_embedding(face_img):
    """Extract embedding using DeepFace ArcFace."""
    from deepface import DeepFace
    try:
        embedding = DeepFace.represent(face_img, model_name='ArcFace', detector_backend='retinaface', enforce_detection=False)
        return np.array(embedding[0]['embedding'])
    except Exception as e:
        print(f"Error extracting embedding: {e}")
        return None

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)

    # Perform YOLO-based face detection
    results = yolo.predict(frame, conf=0.4)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            face = frame[y1:y2, x1:x2]
            (fH, fW) = face.shape[:2]

            # Skip small faces
            if fW < 20 or fH < 20:
                continue

            # Extract embedding
            embedding = get_embedding(face)
            if embedding is None:
                continue  # Skip if embedding extraction fails

            # Perform recognition
            preds = recognizer.predict_proba([embedding])[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # If recognition confidence is low, classify as "Unknown"
            if proba * 100 < 70:
                name = "Unknown"

            # Draw bounding box and recognition text
            text = f"{name} : {proba * 100:.2f}%"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            print(f"Recognized: {name}, Probability: {proba * 100:.2f}%")

    # Show the output frame
    cv2.imshow("YOLO Face Recognition", frame)

    if cv2.getWindowProperty("YOLO Face Recognition", cv2.WND_PROP_VISIBLE) < 1:  # Close button clicked
        break

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC key to quit
        break

# Cleanup
cam.release()
cv2.destroyAllWindows()

[INFO] Loading YOLO model...
[INFO] Loading face recognizer...
[INFO] Loading stored face embeddings...
[INFO] Starting video stream...


d:\PythonProject\Face_Recognition_DL\HogEnv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\PythonProject\Face_Recognition_DL\HogEnv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



0: 480x640 1 face, 70.9ms
Speed: 4.0ms preprocess, 70.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

Recognized: Sayan_FR, Probability: 81.81%

0: 480x640 1 face, 222.5ms
Speed: 44.8ms preprocess, 222.5ms inference, 25.0ms postprocess per image at shape (1, 3, 480, 640)
Recognized: Unknown, Probability: 41.16%

0: 480x640 1 face, 102.0ms
Speed: 3.5ms preprocess, 102.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Recognized: Unknown, Probability: 54.51%

0: 480x640 1 face, 92.0ms
Speed: 3.1ms preprocess, 92.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Recognized: Unknown, Probability: 39.40%

0: 480x640 1 face, 89.4ms
Speed: 2.5ms preprocess, 89.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Recognized: Unknown, Probability: 37.07%

0: 480x640 1 face, 80.4ms
Speed: 5.7ms preprocess, 80.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Recognized: Unknown, Probability: 35.81%
